In [1]:
# self-generated similar design, PEERS semantic matrix
# simplified! no repetition
import pandas as pd
import numpy as np
import pickle

In [2]:
# 1638 words
with open("../wordpools/wasnorm_wordpool.txt") as f:
    items = f.readlines()
items = [i.replace("\n","") for i in items]
items = np.array(items)

In [3]:
simu_sess_num = 1000
nlist = 6
n = 12
wordpool = np.arange(1,1639)
pos_rep = [1,3,5]

In [4]:
df_study = pd.DataFrame()
df_test = pd.DataFrame()
rng = np.random.default_rng(seed=42)

for sess in range(simu_sess_num):

    all_words = rng.choice(wordpool, nlist*2*n, replace = False)

    for lst in range(nlist):

        tmp_df_study = pd.DataFrame()
        tmp_df_test1 = pd.DataFrame()
        tmp_df_test2 = pd.DataFrame()
        
        # choose present words
        pres_words = all_words[lst*2*n:(lst+1)*2*n]
        pres_words = np.reshape(pres_words, (n,2))
        pair_idx = np.arange(1,13) + lst * n + sess * nlist * n
        pres_items = pres_words
        
        tmp_df_study['study_itemno1'] = pres_items[:,0]
        tmp_df_study['study_itemno2'] = pres_items[:,1]
        tmp_df_study['study_item1'] = items[pres_items[:,0]-1]
        tmp_df_study['study_item2'] = items[pres_items[:,1]-1]
        tmp_df_study['list'] = lst
        tmp_df_study['session'] = sess
        df_study = pd.concat([df_study, tmp_df_study])
        
        # choose test words
        test_order = rng.permutation([[1,1]] * int(n/4) + [[1,2]] * int(n/4) + [[2,1]] * int(n/4) + [[2,2]] * int(n/4))
        test1_order = test_order[:,0]
        test2_order = test_order[:,1]
        pres_words = np.array(pres_words)
        test1_probe = np.array([pres_words[i,test1_order[i]-1] for i in range(n)])
        test1_ans = np.array([pres_words[i,2-test1_order[i]] for i in range(n)])
        test2_probe = np.array([pres_words[i,test2_order[i]-1] for i in range(n)])
        test2_ans = np.array([pres_words[i,2-test2_order[i]] for i in range(n)])
        
        # test
        probe_ans_rep = list(zip(test1_probe, test1_ans, test1_order, pair_idx))
        probe_ans_rep = rng.permutation(probe_ans_rep)
        test1_probe, test1_ans, test1_order, test1_idx = zip(*probe_ans_rep)
        test1_probe, test1_ans, test1_order, test1_idx = np.array(test1_probe), np.array(test1_ans), np.array(test1_order), np.array(test1_idx)
        
        tmp_df_test1['test_itemno1'] = test1_probe
        tmp_df_test1['test_itemno2'] = -1
        tmp_df_test1['test_item'] = items[test1_probe-1]
        tmp_df_test1['correct_ans'] = test1_ans
        tmp_df_test1['order'] = test1_order
        tmp_df_test1['rep'] = 1
        tmp_df_test1['test'] = 1
        tmp_df_test1['list'] = lst
        tmp_df_test1['session'] = sess
        tmp_df_test1['test_pos'] = np.arange(1,13)
        tmp_df_test1['pair_idx'] = test1_idx

        probe_ans_rep = list(zip(test2_probe, test2_ans, test2_order, pair_idx))
        probe_ans_rep = rng.permutation(probe_ans_rep)
        test2_probe, test2_ans, test2_order, test2_idx = zip(*probe_ans_rep)
        test2_probe, test2_ans, test2_order, test2_idx = np.array(test2_probe), np.array(test2_ans), np.array(test2_order), np.array(test2_idx)

        tmp_df_test2['test_itemno1'] = test2_probe
        tmp_df_test2['test_itemno2'] = -1
        tmp_df_test2['test_item'] = items[test2_probe-1]
        tmp_df_test2['correct_ans'] = test2_ans
        tmp_df_test2['order'] = test2_order
        tmp_df_test2['rep'] = 1
        tmp_df_test2['test'] = 2
        tmp_df_test2['list'] = lst
        tmp_df_test2['session'] = sess
        tmp_df_test2['test_pos'] = np.arange(13,25)
        tmp_df_test2['pair_idx'] = test2_idx
        df_test = pd.concat([df_test,tmp_df_test1, tmp_df_test2])

df_study.reset_index(inplace = True, drop=True)
df_test.reset_index(inplace = True, drop=True)  

In [5]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,list,session
0,120,459,BELT,DIAPER,0,0
1,1315,1260,SKETCH,SEAFOOD,0,0
2,357,723,COMPANION,HOSTESS,0,0
3,1551,1202,VEAL,ROCKET,0,0
4,1081,1066,PLANE,PIGMENT,0,0
...,...,...,...,...,...,...
71995,306,793,CHOCOLATE,LADDER,5,999
71996,846,1117,LOVER,POWDER,5,999
71997,1102,1048,PONY,PERCH,5,999
71998,658,903,GRAVE,MOAT,5,999


In [6]:
df_test.head(48)

,test_itemno1,test_itemno2,test_item,correct_ans,order,rep,test,list,session,test_pos,pair_idx
0,51,-1,ASTRONAUT,1083,2,1,1,0,0,1,8
1,1202,-1,ROCKET,1551,2,1,1,0,0,2,4
2,979,-1,OPERATOR,142,1,1,1,0,0,3,11
3,134,-1,BLADE,105,1,1,1,0,0,4,9
4,1081,-1,PLANE,1066,1,1,1,0,0,5,5
5,357,-1,COMPANION,723,1,1,1,0,0,6,3
6,831,-1,LITERATURE,1435,2,1,1,0,0,7,7
7,1188,-1,RIB,641,2,1,1,0,0,8,12
8,1260,-1,SEAFOOD,1315,2,1,1,0,0,9,2
9,12,-1,AIRPLANE,226,1,1,1,0,0,10,10


In [7]:
# save
with open('simu6b_data/simu6b_design.pkl', 'wb') as outp:
    pickle.dump(df_study, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(df_test, outp, pickle.HIGHEST_PROTOCOL)